# FiftyOne 脚本

In [1]:
import numba, llvmlite, umap

print("numba:", numba.__version__)
print("llvmlite:", llvmlite.__version__)
print("umap:", umap.__version__)


/home/tianqi/miniconda3/envs/fiftyone/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


numba: 0.59.1
llvmlite: 0.42.0
umap: 0.5.6


In [2]:
RESET_DATASET = True

In [3]:
import fiftyone as fo

dataset_name = fo.list_datasets()[0] if not RESET_DATASET else None
if dataset_name is None:
    session = fo.launch_app(port=5151)
else:
    dataset = fo.load_dataset(dataset_name)
    print(f"Default dataset name: {dataset_name}")
    session = fo.launch_app(dataset,port=5151)

Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port


In [4]:
import fiftyone as fo
# 获取所有dataset
datasets = fo.list_datasets()
display(datasets)

['jeff_0613-0624_04_ok',
 'lloyd_0715-0729_04_ok',
 'ms1_0605-0621_40_ok',
 'ms1_0710-0726_36_ok',
 'ms1_0726-0809_11_ok',
 'ms1_0809-0823_34_ok',
 'ms2_0726-0809_13_ok',
 'ms2_0809-0823_10_ok',
 'sw1_0605-0613_07_ok',
 'sw1_0711-0725_03_ok',
 'sw2_0808-0823_04_ok']

In [5]:
import fiftyone.core.labels as fol
import json
from collections import defaultdict
from datetime import datetime
from pathlib import Path
import fiftyone as fo


def attach_dot_to_fiftyone(
    dataset: fo.Dataset,
    coco_json_path: Path,
    dot_field: str,
):
    """
    根据 COCO json 里的标准字段：
      - annotation["attributes"]["dot_detections"]
    给 FiftyOne dataset 加一个新的可视化字段：
      - sample[dot_field]: fo.Detections（所有 dot box）
    """
    # 1. 读 COCO JSON
    with coco_json_path.open("r", encoding="utf-8") as f:
        coco = json.load(f)

    images = coco["images"]
    annotations = coco["annotations"]

    # image_id -> file_name
    image_id_to_fname = {img["id"]: img["file_name"] for img in images}

    # file_name -> [annotations...] （保持原始顺序）
    anns_by_fname = defaultdict(list)
    for ann in annotations:
        fname = image_id_to_fname[ann["image_id"]]
        anns_by_fname[fname].append(ann)

    # skeleton（跟 categories 里的 keypoints / skeleton 一致）
    dataset.default_skeleton = fo.KeypointSkeleton(
        labels=["h", "lp", "rp"],
        edges=[[0, 1], [0, 2]],
    )

    # 2. 按 sample 遍历，把 pose / dot 加进去
    for sample in dataset:
        fname = Path(sample.filepath).name
        anns = anns_by_fname.get(fname, [])
        if not anns:
            continue

        W = sample.metadata.width
        H = sample.metadata.height

        dot_dets_list = []

        for ann in anns:
            # -------- attributes.dot_detections -> fo.Detections --------
            attrs = ann.get("attributes") or {}
            dot_list = attrs.get("dot_detections") or []
            for dot in dot_list:
                # bbox: [x, y, w, h] 像素坐标
                x, y, w, h = dot["bbox"]
                rel_box = [
                    x / W,
                    y / H,
                    w / W,
                    h / H,
                ]
                det = fol.Detection(
                    bounding_box=rel_box,
                    confidence=dot.get("score", None),
                    label=str(
                        dot.get("category_id", "")
                    ),  # 这里用 category_id，或者直接写 "dot"
                )
                dot_dets_list.append(det)

        # 3. 挂到 sample 上
        if dot_dets_list:
            sample[dot_field] = fol.Detections(detections=dot_dets_list)

        sample.save()

    print(f"✅ 已将 dot 检测框写入字段: {dot_field}")


def extract_time_info(file_name: str) -> datetime:
    time_info = "_".join(file_name.split("_")[:-1])
    dt = datetime.strptime(time_info, "%m%d_%H%M")
    return dt.replace(year=2024)  # 假设年份为2024年


def extract_focus_info(file_name: str) -> str:
    return file_name.split("_")[-1].split(".")[0]

In [6]:
import fiftyone as fo

def add_tags_to_all_labels(
    dataset: fo.Dataset,
    bool_attr: str,
    tag_name: str | None = None,
    mode: str = "both",
):
    """
    给 label 添加 tag，当 label.<bool_attr> 为 True 时打上 tag。

    参数:
        bool_attr:   label 上的布尔属性名，例如 "is_final_swd"
        tag_name:    标签名，默认用 bool_attr
        mode:        "keypoints", "detections", 或 "both"
    """
    if tag_name is None:
        tag_name = bool_attr

    mode = mode.lower()
    assert mode in ("keypoints", "detections", "both"), "mode 必须是 keypoints/detections/both"

    label_fields = dataset._get_label_fields()
    print("所有 label 字段:", label_fields)

    for field in label_fields:
        first_label = dataset.first()[field]

        # 判断这个字段包含 keypoints 或 detections
        is_kp = hasattr(first_label, "keypoints")
        is_det = hasattr(first_label, "detections")

        # 根据 mode 决定是否处理该字段
        if mode == "keypoints" and not is_kp:
            continue
        if mode == "detections" and not is_det:
            continue
        if mode == "both" and not (is_kp or is_det):
            continue

        for sample in dataset:
            labels_layer = sample[field]
            if not labels_layer:
                continue

            # 选择对应的 label 列表
            if is_kp:
                labels = labels_layer.keypoints
            else:
                labels = labels_layer.detections

            changed = False

            for label in labels:
                if not hasattr(label, bool_attr):
                    continue
                if not getattr(label, bool_attr):
                    continue

                if label.tags is None:
                    label.tags = []
                if tag_name not in label.tags:
                    label.tags.append(tag_name)
                    changed = True

            if changed:
                sample[field] = labels_layer
                sample.save()

    print(f"✅ 已根据 `{bool_attr}` 添加 tag: `{tag_name}` (mode={mode})")


In [7]:
import fiftyone.utils.coco as fouc  
from pathlib import Path
from fiftyone import ViewField as F


subdir_path = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try")
subdir_name = "00_try"
# subdir_path = Path("/home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/01_16mp_2024_pipeline_data/ms1_0726-0809_11_ok")
# subdir_name = "ms1_0726-0809_11_ok"

if subdir_name in fo.list_datasets() and RESET_DATASET:
    fo.delete_dataset(subdir_name) 
print(f"subdir_path: {subdir_path}, subdir_name: {subdir_name}")


dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    name=f"{subdir_name}",
    data_path=subdir_path / "raw_data",
    labels_path=subdir_path / "output" / "01_swd_seg_results_coco.json",
    label_field="01_swd_seg_results_coco",
    label_types="detections",
)

# putin rest no annotation image data
dataset.merge_dir(  
    dataset_dir=subdir_path / "raw_data",  
    dataset_type=fo.types.ImageDirectory,  
    skip_existing=True,  # 跳过已存在的样本  
    insert_new=True,     # 插入新样本  
)

fouc.add_coco_labels(
    dataset,
    label_field="02_combined_annotations_dedup",
    labels_or_path=str(subdir_path / "output" / "02_combined_annotations_dedup.json"),
    categories={1: "swd"},
    label_type="detections",
)


fouc.add_coco_labels(
    dataset,
    label_field="03_coco_with_pose_dot_keypoints",
    labels_or_path=str(subdir_path / "output" / "03_coco_with_pose_dot.json"),
    categories={1: "swd"},
    label_type="keypoints",
)

fouc.add_coco_labels(
    dataset,
    label_field="03_coco_with_pose_dot_detections",
    labels_or_path=str(subdir_path / "output" / "03_coco_with_pose_dot.json"),
    categories={1: "swd"},
    label_type="detections",
)


attach_dot_to_fiftyone(
    dataset=dataset,
    coco_json_path=subdir_path / "output" / "03_coco_with_pose_dot.json",
    dot_field="04_dot_boxes",
)

add_tags_to_all_labels(dataset, "is_final_swd", "fswd","detections")

# 添加时间和focus信息,通过file_name获取， 0606_0617_760.jpg 0606_0617表示时间信息 760表示焦点距离
for sample in dataset:
    file_name = sample.filepath.split("/")[-1]
    sample["Date"] = extract_time_info(file_name)
    sample["focus"] = extract_focus_info(file_name)
    sample.save()

print("✅ 已添加时间和焦点信息")

# 6. 建立索引  
dataset.create_index("Date")  
dataset.create_index("focus")  

subdir_path: /home/tianqi/D/01_Projects/01_swd/02_code/pipeline/ultralytics_ty/_ty/01_data/00_test/00_try, subdir_name: 00_try
 100% |█████████████████████| 4/4 [99.8ms elapsed, 0s remaining, 40.1 samples/s]     
 100% |█████████████████████| 4/4 [12.0ms elapsed, 0s remaining, 334.6 samples/s]    
Converting existing index 'filepath' to unique on dataset '2025.12.03.12.31.02.249377'
Converting existing index 'filepath' to unique on dataset '00_try'
✅ 已将 dot 检测框写入字段: 04_dot_boxes
所有 label 字段: ['01_swd_seg_results_coco', '02_combined_annotations_dedup', '03_coco_with_pose_dot_keypoints', '03_coco_with_pose_dot_detections', '04_dot_boxes']
✅ 已根据 `is_final_swd` 添加 tag: `fswd` (mode=detections)
✅ 已添加时间和焦点信息


'focus'

In [8]:
for key in dataset.list_brain_runs():
    print(f"Deleting existing brain run: {key}")
    # dataset.delete_brain_run(key)

In [9]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob

label_field = "01_swd_seg_results_coco"

# embedding模型
models = ["clip-vit-base32-torch", "dinov2-vits14-torch", "resnet50-imagenet-torch", "mobilenet-v2-imagenet-torch"]
# 降维方法
dim_reduction_methods = ["umap", "tsne", "pca"]

embeddings_fields = ["emb_clip", "emb_dinov2", "emb_resnet50", "emb_mobilenet"]

datasets = ["00_try"]

# 清理已有的 brain runs，避免冲突
for dataset_name in datasets:
    dataset = fo.load_dataset(dataset_name)
    for key in dataset.list_brain_runs():
        dataset.delete_brain_run(key)


for model_name, emb_field in zip(models, embeddings_fields):
    model = foz.load_zoo_model(model_name)

    for dataset_name in datasets:
        print(f"Dataset: {dataset_name}")
        dataset = fo.load_dataset(dataset_name)

        # 1) 对每个 ann 直接算 patch embedding（按 bbox/mask 裁剪，不导出图片）
        dataset.compute_patch_embeddings(
            model,
            patches_field=label_field,   # 关键：按这个字段里的 bbox/mask 作为 patch
            embeddings_field=emb_field,      # embedding 存在每个 ann 的 .emb 里
        )

        for method in dim_reduction_methods:
            brain_key = f"patches_{model_name.split('-')[0]}_{method}_v1"

            # 2) 对所有 patch 做 降维 可视化
            fob.compute_visualization(
                dataset,
                patches_field=label_field,   # 告诉 brain 这是 patch 字段
                embeddings=emb_field,            # 用上一步算好的 embedding 字段
                method=method,                # 先用 pca，规避 umap/numba 问题
                seed=51,
                brain_key=brain_key,  # 每个 dataset 自己有一份同名 brain_key 就行
            )

Dataset: 00_try
 100% |█████████████████████| 4/4 [2.6s elapsed, 0s remaining, 1.6 samples/s]      
Generating visualization...
UMAP(n_jobs=1, random_state=51, verbose=True)
Wed Dec  3 12:31:09 2025 Construct fuzzy simplicial set
Wed Dec  3 12:31:09 2025 Finding Nearest Neighbors


/home/tianqi/miniconda3/envs/fiftyone/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/tianqi/miniconda3/envs/fiftyone/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Wed Dec  3 12:31:10 2025 Finished Nearest Neighbor Search
Wed Dec  3 12:31:11 2025 Construct embedding


Epochs completed: 100%| ██████████ 500/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Wed Dec  3 12:31:11 2025 Finished embedding


...

Epochs completed:  98%| █████████▊ 490/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs


Epochs completed: 100%| ██████████ 500/500 [00:00]

Wed Dec  3 12:31:35 2025 Finished embedding
Generating visualization...


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 333 samples in 0.000s...
[t-SNE] Computed neighbors for 333 samples in 0.027s...
[t-SNE] Computed conditional probabilities for sample 333 / 333
[t-SNE] Mean sigma: 2.082960
[t-SNE] Computed conditional probabilities in 0.007s
[t-SNE] Iteration 50: error = 64.2189102, gradient norm = 0.2931502 (50 iterations in 0.066s)
[t-SNE] Iteration 100: error = 63.3232384, gradient norm = 0.2744624 (50 iterations in 0.020s)
[t-SNE] Iteration 150: error = 63.1111145, gradient norm = 0.3054615 (50 iterations in 0.020s)
[t-SNE] Iteration 200: error = 64.3853226, gradient norm = 0.2803025 (50 iterations in 0.017s)
[t-SNE] Iteration 250: error = 64.6651459, gradient norm = 0.2768654 (50 iterations in 0.017s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 64.665146
[t-SNE] Iteration 300: error = 0.7335223, gradient norm = 0.0033968 (50 iterations in 0.015s)
[t-SNE] Iteration 350: error = 0.6853495, gradient norm = 0.0031633 (